## Imports

In [1]:
from about_data import * 
from clean import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Local file paths 

In [2]:
data_file = '/Users/Winnifred/Desktop/Capstone/ICPSR_20240_RAWDATA/DS0001/20240-0001-Data.tsv'
txt_path = '/Users/Winnifred/Desktop/Capstone/diagnosis_capstone/data/feature_group_file_names.txt'
csv_root_path = '/Users/Winnifred/Desktop/Capstone/diagnosis_capstone/data/feature_name_data/'

## Data

In [10]:
messy_df = pd.read_csv(data_file, sep='\t', low_memory=False)

In [4]:
universal_features, nonuniversal_features = get_universals(txt_path, csv_root_path)

In [11]:
universal_df = pd.DataFrame()

In [12]:
for feature in good_features: 
    if feature not in suicidality_features: 
        universal_df[feature] = messy_df[feature]

In [ ]:
feature_uniques = {}
for feature in good_features:
    feature_uniques[feature] = list(pd.unique(clean_df[feature]))

In [ ]:
feature_uniques['V01638']

## Cleaning Functions Test Area

### Getting the universal features

In [ ]:
universal_features, nonuniversal_features = get_universals(txt_path, csv_root_path)

In [6]:
good_features = list(universal_features.keys())

In [ ]:
good_features

### Finding features with a specific range of responses

In [ ]:
# find the maximum for a feature 
def feature_max(feature):
    x = feature_uniques[feature]
    # x is [' ', '-8', '-9', '1', '5']
    sorted_x = sorted(x, reverse=True)
    # sorted_x is ['5', '1', '-9', '-8', ' ']
    no_space = sorted_x[0:-1]
    # no_space is ['5', '1', '-9', '-8']
    range_list = []
    for i in no_space: 
        range_list.append(float(i))
    # range_list is [5, 1, -9, -8]
    # min(range_list) is -9
    # max(range_list) is 5
    return max(range_list)

In [ ]:
features_max5 = []
for feature in good_features: 
    if feature_max(feature) <= 5.0: 
        features_max5.append(feature)

In [ ]:
my_clean_df = pd.DataFrame()
for item in features_max5: 
    my_clean_df[item] = messy_df[item]

### These features appear in all three surveys and have a max value of 5

In [ ]:
features_max5

What groups do they belong to? 

In [24]:
full_dict = setup(txt_path, csv_root_path)

In [ ]:
groups = []
descriptions = []
for feature in features_max5:
    groups.append(get_group(feature, full_dict))
    descriptions.append(get_descriptions(feature, full_dict))
    

In [ ]:
def get_group(feature, full_dict):
    for akey, apair in full_dict.items():
        for bkey, bpair in apair.items(): 
            for ckey, cpair in bpair.items(): 
                if feature == ckey: 
                    return bkey

In [ ]:
get_group('V01681', full_dict)

In [ ]:
def get_descriptions(feature, full_dict):
    for akey, apair in full_dict.items():
        for bkey, bpair in apair.items(): 
            for ckey, cpair in bpair.items(): 
                if feature == ckey: 
                    return cpair[0]    

In [ ]:
get_descriptions('V01681', full_dict)

In [ ]:
set(groups)

In [ ]:
descriptions

## IMPORTANT TO DO ITEMS
- build a basic model

## Things I've Done Today: 
- Look at distribution of features between categories of symptoms and diagnosis after cleaning (i.e. what proportion of each group made it through the universals test AND max_5)

In [56]:
universal_df.head()

,V01626,V01627,V01628,V01629,V01630,V01631,V01632,V01633,V01634,V01635,...,V08501,V08500,V08553,V09045,V09048,V07750,V07748,V07899,V09043,V09046
0,,,,,,,,,,,...,5,5,5,,,5,5,5,,
1,,,,,,,,,,,...,5,5,5,,,5,5,5,,
2,,,,,,,,,,,...,5,5,5,,,5,5,5,,
3,,,,,,,,,,,...,5,5,5,,,5,5,5,,
4,,,,,,,,,,,...,5,5,5,,,5,5,5,,


In [16]:
ideation_df = pd.DataFrame()
ideation_df['suicide_ideation'] = universal_df['V01993']

In [19]:
mask = {'5': 0, '1': 1, '-9': 0, '-8': 0, ' ': None}

In [21]:
ideation_df['suicide_ideation'] = ideation_df['suicide_ideation'].map(mask)

In [27]:
screening_dict = full_dict['Screening']['Screening']

In [30]:
# get a list of all the good features from screening section
good_screen_features = []
for feature in screening_dict.keys(): 
    if feature in good_features: 
        good_screen_features.append(feature)

In [57]:
# add the good screening features to the dataframe
for feature in good_screen_features: 
    ideation_df[feature] = universal_df[feature]

In [58]:
# change ints to string through dataframe
for feature in good_screen_features: 
    ideation_df[feature] = ideation_df[feature].map(str_to_int)

In [61]:
df = ideation_df.dropna()

In [170]:
df.head()

,suicide_ideation,V00232,V00233,V00234,V00239,V00252,V00253,V00254,V00255,V00256,...,V00725,V00728,V00729,V00730,V00731,V00738,V00739,V00740,V00741,V00742
0,0.0,3.0,3.0,3.0,0.0,5.0,5.0,5.0,5.0,5.0,...,0.0,5.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,5.0
2,0.0,3.0,3.0,2.0,0.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,5.0
3,0.0,2.0,2.0,1.0,0.0,5.0,5.0,5.0,5.0,5.0,...,5.0,1.0,1.0,5.0,5.0,5.0,1.0,0.0,0.0,1.0
5,1.0,1.0,3.0,1.0,0.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,1.0
6,0.0,1.0,2.0,2.0,0.0,1.0,1.0,5.0,5.0,5.0,...,1.0,5.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,5.0


In [43]:
str_to_int = {}
for number in range(150):
    str_to_int[str(number)] = number

In [55]:
str_to_int[' '] = 0

In [ ]:
np_array = clean_df.as_matrix()

In [ ]:
np_array

In [ ]:
my_clean_df

In [9]:
suicidality_features = ['V01992','V01994','V01995','V01996','V01997','V01998','V01999','V02000','V02044','V02045','V02001','V02002','V02003','V02004','V02005','V02009','V02010','V02023','V02024','V02025','V02026','V02027','V02028','V02029','V02030','V02031','V02032','V02033','V02034','V02035','V02036', 'V02037', 'V02041', 'V02042']

In [ ]:
label_ideation = 'V01993'

In [83]:
my_y = df['suicide_ideation']

In [84]:
my_X = df.loc[:, df.columns != 'suicide_ideation']

In [134]:
def print_scores(y_test, predictions):
    print('Accuracy:', accuracy_score(y_test, predictions))
    print('Precision:', precision_score(y_test, predictions))
    print('Recall:', recall_score(y_test, predictions))
    return None

## Model Experimentation Area

Let's try to do some logistic regression on suicide ideation...

In [103]:
X = my_X.as_matrix()
y = my_y.as_matrix()

In [130]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [111]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(10506, 59) (10506,)
(4503, 59) (4503,)


In [112]:
logreg = linear_model.LogisticRegression()

In [113]:
model = logreg.fit(X_train, y_train)

In [114]:
predictions = logreg.predict(X_test)

In [122]:
print('Score:', model.score(X_test, y_test))

Score: 0.874083944037


In [135]:
print_scores(y_test, predictions)

Accuracy: 0.874083944037
Precision: 0.520710059172
Recall: 0.15331010453


Woooooow that precision is like flipping a coin and that recall is horrendous!

What can we do about that? Let's start by diving into the features...maybe even culling for features I am particularly interested in. Also, re-assess how I did the cleaning. 

But first, let's try a random forest and see if that does any better. 

## Random Forest Experimentation Area

In [136]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [137]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [138]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [139]:
print(clf.feature_importances_)

[ 0.          0.          0.          0.03094058  0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.10645756  0.05410021
  0.          0.06733384  0.          0.2321134   0.07302103  0.01547272
  0.          0.01589367  0.01266059  0.          0.02689626  0.06914118
  0.          0.01253428  0.10405639  0.10414674  0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.05381668  0.          0.          0.          0.          0.01819823
  0.          0.          0.          0.00321664  0.          0.          0.        ]


In [161]:
feature_importances = list(clf.feature_importances_)

In [159]:
print(len(feature_importances),len(good_screen_features))

59 59


In [168]:
imprtnt_gd_scrn_ftrs = []
for idx, feature in enumerate(good_screen_features): 
    if feature_importances[idx] > 0: 
        imprtnt_gd_scrn_ftrs.append(feature)

In [166]:
feature_importances[2] > 0

False

In [169]:
imprtnt_gd_scrn_ftrs

['V00239',
 'V00684',
 'V00688',
 'V00686',
 'V00690',
 'V00691',
 'V00692',
 'V00695',
 'V00696',
 'V00708',
 'V00709',
 'V00705',
 'V00706',
 'V00707',
 'V00727',
 'V00729',
 'V00739']